In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import operator
from math import log

In [14]:
def infoEnt(dataset, classList):
    '''
    计算给定数据集的信息熵
    :param dataset:输入的数据集
    :return: 返回信息熵
    '''
    
    # 计算出数据集的总数
    numEntries = len(dataset)
    
    # 用来统计分类标签
    labelCounts = dataset[dataset.columns[-1]].apply(pd.value_counts).count()
        
    # 默认的信息熵
    infoEntropy = 0.0
    
    for key in range(len(labelCounts)):
        # 计算出当前分类标签占总标签的比例数
        prob = float(labelCounts[key]) / numEntries
        
        # 以2为底求对数
        infoEntropy -= prob * log(prob, 2)
    
    return infoEntropy

In [15]:
def splitDataset(dataset, axis, value):
    '''
    对某个特征进行划分后的数据集
    :param dataset:数据集
    :param axis:给定特征值的坐标
    :param value:给定特征值满足的条件，只有给定特征值等于这个value时才会返回
    :return:返回剩余数据集
    '''
    
    # 创建一个新的列表，防止对原来的列表进行修改
    retDataset = dataset
    
    # 遍历整个数据集
    droplist = []
    for i in range(len(dataset)):
        # 如果给定特征值等于想要的特征值
        if retDataset.iloc[i, axis] != value:
            droplist.append(i)
    retDataset = retDataset.drop(droplist)
    print('retDataset:', retDataset)
    return retDataset

In [16]:
def bestFeatureSplit(dataset, labels, classList):
    '''
    最优属性划分
    :param dataset:输入需要划分的数据集
    :param labels:特征标签
    :return:返回最优划分属性的下标
    '''
    
    # 得到数据的特征值总数
    numFeatures = len(dataset.columns) - 1
    
    # 计算出基础信息熵
    baseEntropy = infoEnt(dataset, classList)
    print('baseEntropy:', baseEntropy)
    
    # 基础信息增益为0.0
    bestInfoGain = 0.0
    
    # 最好的特征值
    bestFeature = -1
    
    # 对每个特征值进行求信息熵
    for i in range(numFeatures):
        # 得到数据集中所有的当前特征值列表
        featList = dataset[dataset.columns[i]]
        print('featList', featList)
        # 将当前特征唯一化，也就是说当前特征值中共有多少种
        uniqueVals = set(featList)
        
        # 新的熵，代表当前特征值的熵
        newEntropy = 0.0
        
        # 遍历现有的特征的可能性
        for value in uniqueVals:
            # 在全部数据集的当前特征位置上，找到特征值等于当前值的集合
            subDataset = splitDataset(dataset, i, value)
            
            # 拿到所有数据集的分类标签
            subclassList = subDataset[subDataset.columns[-1]]
            
            # 计算出权重
            prob = len(subDataset) / float(len(dataset))
            print(prob)
            
            # 计算出当前特征值的熵
            newEntropy += prob * infoEnt(subDataset, subclassList)
            
        print(labels[i],'Entropy:',newEntropy)
        
        # 计算出“信息增益”
        infoGain = baseEntropy - newEntropy
        print('infoGain:',infoGain)
        
        # print('当前特征值为：' + labels[i] + '，对应的信息增益值为：' + str(infoGain))
        
        # 如果当前的信息增益比原来的大
        if infoGain > bestInfoGain:
            # 最好的信息增益
            bestInfoGain = infoGain
            # 新的最好的用来划分的特征值
            bestFeature = i
    
    # rint('信息增益最大的特征为：' + labels[bestFeature])
    return bestFeature

In [17]:
def majorClass(classList):
    '''
    对叶节点的分类结果进行划分，按照数量大小
    找到次数最多的类别标签
    :param classList:叶节点上的样本数量
    :return:返回叶节点划分结果
    '''
    
    # 用来统计标签的票数
    classCount = dataset[dataset.columns[-1]].apply(pd.value_counts).count()
        
    # 从小到大排序
    sortedClassCount = sorted(classCount.items(), key = operator.itemgetter(1), reverse = True)
    
    # 返回次数最多的标签
    return sortedClassCount[0][0]
    
    

In [18]:
def judgeEqualLabels(dataset):
    '''
    判断数据集的各个属性集是否完全一致
    :param dataset:数据集
    :return:True/False
    '''
    
    # 计算出样本集中共有多少个属性，最后一个为类别
    feature_len = len(dataset.columns) - 1
    
    # 计算出共有多少个数据
    data_len = len(dataset)
    
    # 标记每个属性中第一个属性值是什么
    first_feature = ''
    
    # 各个属性集是否完全一致
    is_equal = True
    
    # 遍历全部属性
    for i in range(feature_len):
        # 得到第一个样本的第i个属性
        first_feature = dataset.iloc[0, i]
        
        # 与样本集中所有的数据进行对比，看看在该属性上是否都一致
        for j in range(1, data_len):
            # 如果发现不相等的，则直接返回False
            if first_feature != dataset.iloc[j ,i]:
                return False
    
    return is_equal
    
    

In [19]:
def createTree(dataset, labels):
    '''
    创建决策树
    :param dataset: 数据集
    :param labels:特征标签
    :return: 返回决策树字典
    '''
    print('dataset:', dataset)
    
    # 拿到所有数据集的分类标签
    classList = dataset.iloc[:, -1]
    
    # 统计第一个标签出现的次数，与总标签个数比较，如果相等则说明当前列表中全部都是一种标签，此时停止划分
    if np.sum(np.char.count(classList.iloc[0],classList)) == len(classList):
        return classList.iloc[0]
    
    # 计算第一行有多少个数据，如果只有一个的话说明所有的特征属性都遍历完了，剩下的一个就是类别标签，或者所有的样本在全部属性上都一致
    if len(dataset.columns) == 1 or judgeEqualLabels(dataset):
        # 返回剩下标签中出现次数较多的那个
        return majorClass(classList)
    
    # 选择最好的划分特征，得到该特征的下标
    bestFeat = bestFeatureSplit(dataset, labels, classList)
    
    # 得到最好特征的名称
    bestFeatLabel = labels[bestFeat]
    
    # 使用一个字典来存储树结构，分叉处为划分的特征名称
    myTree = {bestFeatLabel:{}}
    
    # 将本次划分的特征值从列表中删除
    subLabels = dataset.columns[:-1].delete(bestFeat)
    print('sublabels:',subLabels)
    
    # 得到当前特征标签的所有可能值
    featValues = dataset[bestFeatLabel]
    
    # 唯一化，去掉重复的特征值
    uniqueVals = set(featValues)
    
    # 遍历所有的特征值
    for value in uniqueVals:
        # 将当前数据集按特征分配子结点splitDataset(subdataset, bestFeat, value)
        subdataset = splitDataset(dataset, bestFeat, value)
        
        # 将本次划分的特征值从列表中删除
        subdataset = subdataset.drop([bestFeatLabel],axis = 1)
        
        # 重排subdataset的index
        subdataset.index = range(len(subdataset))
        
        # 创建子结点
        subTree = createTree(subdataset, subLabels)
        
        # 递归调用，将数据集中该特征等于当前特征值的所有数据划分到当前节点下，递归调用时需要先将当前的特征去除掉
        myTree[bestFeatLabel][value] = subTree
    return myTree
    

In [20]:
def createDataSet():
    '''
    创建测试的数据集
    '''
    dataset = pd.DataFrame({
        'Appearance':['Good', 'Good', 'Great', 'Ah', 'Ah', 'Ah', 'Great', 'Good', 'Good', 'Ah', 'Good', 'Great', 'Great', 'Ah'],
        'Income':['Low', 'Low', 'Low', 'Good', 'Great', 'Great', 'Great', 'Good', 'Great', 'Good', 'Good', 'Good', 'Low', 'Good'],
        'Age':['Older', 'Older', 'Older', 'Older', 'Younger', 'Younger', 'Younger', 'Older', 'Younger', 'Younger', 'Younger', 'Older', 'Younger', 'Older'],
        'Profession':['Steady', 'Unstable', 'Steady', 'Steady', 'Steady', 'Unstable', 'Unstable', 'Steady', 'Steady', 'Steady', 'Unstable', 'Unstable', 'Steady', 'Unstable'],
        'Is_soninlaw_Welcome':['N', 'N', 'Y', 'Y', 'Y', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'N']
    }
    )

    # 特征值列表
    labels = dataset.columns[:-1]
    
    # 特征对应的所有可能的情况
    labelsFull = {}
    for i in labels:
        labelsFull[i] = set(dataset.loc[:, i])
    
    return dataset, labels, labelsFull

In [21]:
def main():
    dataset, labels, labelsFull = createDataSet()
    datasetFull = dataset[:]
    myTree = createTree(dataset, labels)
    print('myTree:',myTree)

In [22]:
if __name__ == '__main__':
    main()

dataset:    Appearance Income      Age Profession Is_soninlaw_Welcome
0        Good    Low    Older     Steady                   N
1        Good    Low    Older   Unstable                   N
2       Great    Low    Older     Steady                   Y
3          Ah   Good    Older     Steady                   Y
4          Ah  Great  Younger     Steady                   Y
5          Ah  Great  Younger   Unstable                   N
6       Great  Great  Younger   Unstable                   Y
7        Good   Good    Older     Steady                   N
8        Good  Great  Younger     Steady                   Y
9          Ah   Good  Younger     Steady                   Y
10       Good   Good  Younger   Unstable                   Y
11      Great   Good    Older   Unstable                   Y
12      Great    Low  Younger     Steady                   Y
13         Ah   Good    Older   Unstable                   N
baseEntropy: 0.9402859586706309
featList 0      Good
1      Good
2     Great

baseEntropy: 0.9709505944546686
featList 0      Low
1      Low
2     Good
3    Great
4     Good
Name: Income, dtype: object
retDataset:   Income    Age Profession Is_soninlaw_Welcome
0    Low  Older     Steady                   N
1    Low  Older   Unstable                   N
0.4
retDataset:   Income      Age Profession Is_soninlaw_Welcome
2   Good    Older     Steady                   N
4   Good  Younger   Unstable                   Y
0.4
retDataset:   Income      Age Profession Is_soninlaw_Welcome
3  Great  Younger     Steady                   Y
0.2
Income Entropy: 0.4
infoGain: 0.5709505944546686
featList 0      Older
1      Older
2      Older
3    Younger
4    Younger
Name: Age, dtype: object
retDataset:   Income    Age Profession Is_soninlaw_Welcome
0    Low  Older     Steady                   N
1    Low  Older   Unstable                   N
2   Good  Older     Steady                   N
0.6
retDataset:   Income      Age Profession Is_soninlaw_Welcome
3  Great  Younger     Steady 